In [ ]:
import bw2calc as bc
import bw_processing as bwp
import matrix_utils as mu
import numpy as np
import stats_arrays as sa
import seaborn as sb

In [ ]:
coal = 900
steel = 901
iron_mining = 902
steel_bike = 903
iron_ore = 1000

In [ ]:
t_data = np.array([
    1,
    1,
    1,
    1,
    1.1,
    0.2,
    8,
    0.001,
    0.001,
    0.005,
])
t_indices = np.array(
    [
        (coal, coal),
        (steel, steel),
        (iron_mining, iron_mining),
        (steel_bike, steel_bike),
        (iron_mining, steel),
        (coal, steel),
        (steel, steel_bike),
        (steel_bike, iron_mining),
        (steel_bike, coal),
        (steel_bike, steel),
    ], 
    dtype=bwp.INDICES_DTYPE
)
t_uncertainty = np.array([
        (sa.NoUncertainty.id, 1, np.NaN, np.NaN, np.NaN, np.NaN, False),
        (sa.NoUncertainty.id, 1, np.NaN, np.NaN, np.NaN, np.NaN, False),
        (sa.NoUncertainty.id, 1, np.NaN, np.NaN, np.NaN, np.NaN, False),
        (sa.NoUncertainty.id, 1, np.NaN, np.NaN, np.NaN, np.NaN, False),
        (sa.TriangularUncertainty.id, 1.1, np.NaN, np.NaN, 1, 1.25, False), 
        (sa.TriangularUncertainty.id, 0.2, np.NaN, np.NaN, 0.1, 0.5, False),
        (sa.TriangularUncertainty.id, 8, np.NaN, np.NaN, 4, 15, False),
        (sa.NoUncertainty.id, 0.001, np.NaN, np.NaN, np.NaN, np.NaN, False),
        (sa.NoUncertainty.id, 0.001, np.NaN, np.NaN, np.NaN, np.NaN, False),
        (sa.NoUncertainty.id, 0.005, np.NaN, np.NaN, np.NaN, np.NaN, False),
    ], 
    dtype=bwp.UNCERTAINTY_DTYPE
)
t_flip = np.array([False, False, False, False, True, True, True, True, True, True])

In [ ]:
b_data = np.array([1.25])
b_uncertainty = np.array([
        (sa.TriangularUncertainty.id, 1.25, np.NaN, np.NaN, 1, 1.75, False), 
    ], 
    dtype=bwp.UNCERTAINTY_DTYPE
)
b_indices = np.array([
    (iron_ore, iron_mining),
    ], 
    dtype=bwp.INDICES_DTYPE
)

In [ ]:
c_data = np.array([3.141])
c_uncertainty = np.array([
        (sa.NormalUncertainty.id, 3.141, 0.5926535, np.NaN, np.NaN, np.NaN, False), 
    ], 
    dtype=bwp.UNCERTAINTY_DTYPE
)
c_indices = np.array([
    (iron_ore, iron_ore),
    ], 
    dtype=bwp.INDICES_DTYPE
)

We can now create our datapackage, and add the data for all three matrices to it:

In [ ]:
dp_static = bwp.create_datapackage()

In [ ]:
dp_static.add_persistent_vector(
    matrix='technosphere_matrix',
    indices_array=t_indices,
    data_array=t_data,
    flip_array=t_flip,
    distributions_array=t_uncertainty,
)
dp_static.add_persistent_vector(
    matrix='biosphere_matrix',
    indices_array=b_indices,
    data_array=b_data,
    distributions_array=b_uncertainty,
)
dp_static.add_persistent_vector(
    matrix='characterization_matrix',
    indices_array=c_indices,
    data_array=c_data,
    distributions_array=c_uncertainty,
)

In [ ]:
lca = bc.LCA(
    demand={steel_bike: 1},
    data_objs=[dp_static],
    use_distributions=True,
)
lca.lci()
lca.lcia()

In [ ]:
mc_results = [lca.score for _ in zip(range(50), lca)]
sb.displot(np.array(mc_results))